In [142]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [143]:
import numpy as np
import pandas as pd

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout, Conv1D, MaxPooling1D, GlobalAveragePooling1D, LSTM, Activation, GRU, SimpleRNN
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
import keras.callbacks as kcallbacks
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam



#20 normal + 20 anomalous = test / con lai la train
#fwap + http = train

# Path

## Normal

In [144]:
# DATASET_TRAINING = '/content/drive/MyDrive/UIT/HocKy8/KLTN/datasets/dataset1/matrix1/normal/train-normal.csv'
# DATASET_TESTING='/content/drive/MyDrive/UIT/HocKy8/KLTN/datasets/dataset1/matrix1/normal/test_csic2010-normal.csv'
# DATASET_TESTING='/content/drive/MyDrive/UIT/HocKy8/KLTN/datasets/dataset1/matrix1/normal/test_fwaf-normal.csv'
# DATASET_TESTING='/content/drive/MyDrive/UIT/HocKy8/KLTN/datasets/dataset1/matrix1/normal/test_http_param-normal.csv'

## minmax

In [145]:
# DATASET_TRAINING = '/content/drive/MyDrive/UIT/HocKy8/KLTN/datasets/dataset1/matrix1/minmax/train_minmax.csv'
# DATASET_TESTING='/content/drive/MyDrive/UIT/HocKy8/KLTN/datasets/dataset1/matrix1/minmax/test_csic2010-minmax.csv'
# DATASET_TESTING='/content/drive/MyDrive/UIT/HocKy8/KLTN/datasets/dataset1/matrix1/minmax/test_fwaf-minmax.csv'
# DATASET_TESTING='/content/drive/MyDrive/UIT/HocKy8/KLTN/datasets/dataset1/matrix1/minmax/test_http_param-minmax.csv'

## binary

In [146]:
DATASET_TRAINING = '/content/drive/MyDrive/UIT/HocKy8/KLTN/detect_attack_by_reinforcement_learning/data/matrix1/binary/train_binary.csv'
# DATASET_TESTING='/content/drive/MyDrive/UIT/HocKy8/KLTN/detect_attack_by_reinforcement_learning/data/matrix1/binary/test_csic2010-binary.csv'
# DATASET_TESTING='/content/drive/MyDrive/UIT/HocKy8/KLTN/detect_attack_by_reinforcement_learning/data/matrix1/binary/test_fwaf-binary.csv'
DATASET_TESTING='/content/drive/MyDrive/UIT/HocKy8/KLTN/detect_attack_by_reinforcement_learning/data/matrix1/binary/test_http_param-binary.csv'

# load data

In [147]:
### Load data
df_train = pd.read_csv(DATASET_TRAINING)
# display(df_train)
# df_test = pd.read_csv(DATASET_TESTING, header=None, index_col=False)
df_test = pd.read_csv(DATASET_TESTING)

df_train = df_train.astype(float)
df_test = df_test.astype(float)

df_train = df_train.sample(frac = 1)
df_test = df_test.sample(frac = 1)

x_train, y_train = df_train.iloc[:, :-1], df_train.iloc[:, [-1]]
x_test, y_test = df_test.iloc[:, :-1], df_test.iloc[:, [-1]]

x_train = x_train/255.0
x_test = x_test/255.0

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1)

y_train = y_train.astype(int)
y_val = y_val.astype(int)
y_test = y_test.astype(int)

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

input_shape = x_train.shape[1]
output_shape = y_train.shape[1]

In [148]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)
print(input_shape, output_shape)

(99954, 38)
(99954, 2)
(11107, 38)
(11107, 2)
(5660, 38)
(5660, 2)
38 2


In [149]:
checkpoint_filepath = '/content/drive/MyDrive/UIT/HocKy8/KLTN/checkpoints/weights.{epoch:02d}-{val_loss:.2f}.h5'
model_checkpoint_callback = kcallbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# CNN


## CNN 1

In [150]:
# ### CNN 1
model = Sequential()
model.add(Conv1D(64, 3, padding="same",activation="relu",input_shape=(input_shape, 1)))
model.add(Conv1D(64, 3, padding="same", activation="relu"))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(output_shape, activation="softmax"))

# define optimizer and objective, compile cnn

model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

# train
early_stop = kcallbacks.EarlyStopping(monitor='val_accuracy', patience=2, verbose=1, mode='auto')
history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=64, callbacks=[model_checkpoint_callback], shuffle=True)

# # loss: 0.2412 - accuracy: 0.8811

Epoch 1/100
1562/1562 [==============================] - 20s 12ms/step - loss: 0.2391 - accuracy: 0.8857 - val_loss: 0.0805 - val_accuracy: 0.9681
Epoch 2/100
1562/1562 [==============================] - 17s 11ms/step - loss: 0.0801 - accuracy: 0.9689 - val_loss: 0.0714 - val_accuracy: 0.9715
Epoch 3/100
1562/1562 [==============================] - 17s 11ms/step - loss: 0.0717 - accuracy: 0.9718 - val_loss: 0.0640 - val_accuracy: 0.9720
Epoch 4/100
1562/1562 [==============================] - 19s 12ms/step - loss: 0.0683 - accuracy: 0.9724 - val_loss: 0.0636 - val_accuracy: 0.9733
Epoch 5/100
1562/1562 [==============================] - 17s 11ms/step - loss: 0.0672 - accuracy: 0.9727 - val_loss: 0.0626 - val_accuracy: 0.9729
Epoch 6/100
1562/1562 [==============================] - 17s 11ms/step - loss: 0.0658 - accuracy: 0.9733 - val_loss: 0.0620 - val_accuracy: 0.9728
Epoch 7/100
1562/1562 [==============================] - 19s 12ms/step - loss: 0.0650 - accuracy: 0.9733 - val_loss: 0

## CNN 2

In [151]:
# ### CNN 2
# model = Sequential()

# model.add(Conv1D(64, 3, activation="relu", padding='same', input_shape=(input_shape, 1)))
# model.add(MaxPooling1D(pool_size=(2)))

# model.add(Flatten())
# model.add(Dense(128, activation="relu"))
# model.add(Dropout(0.5))
# model.add(Dense(output_shape, activation="softmax"))

# # define optimizer and objective, compile cnn
# model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

# early_stop = kcallbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, mode='auto')
# #fit model
# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=20, batch_size=64, callbacks=[model_checkpoint_callback], shuffle=True)

# # # loss: 0.2433 - accuracy: 0.8846 - 128
# # # loss: 0.2384 - accuracy: 0.8884 - 64
# # # loss: 0.2347 - accuracy: 0.8934 - 32

## CNN 3

In [152]:
# ### CNN 3
# model = Sequential()
# model.add(Conv1D(64, 3, padding="same",activation="relu",input_shape=(input_shape, 1)))
# model.add(Conv1D(64, 3, padding="same", activation="relu"))
# model.add(MaxPooling1D(pool_size=(2)))
# model.add(Conv1D(128, 3, padding="same", activation="relu"))
# model.add(Conv1D(128, 3, padding="same", activation="relu"))
# model.add(MaxPooling1D(pool_size=(2)))
# model.add(Flatten())
# model.add(Dense(128, activation="relu"))
# model.add(Dropout(0.5))
# model.add(Dense(output_shape, activation="softmax"))

# # define optimizer and objective, compile cnn
# model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=64, callbacks=[model_checkpoint_callback], shuffle=True)

# # loss: 0.2384 - accuracy: 0.8908

## CNN5

In [153]:
# ### CNN5
# model = Sequential()
# model.add(Conv1D(16, 3, padding="same",activation="relu",input_shape=(input_shape, 1)))
# model.add(Conv1D(64, 3, padding="same", activation="relu"))
# model.add(MaxPooling1D(pool_size=(2)))
# model.add(Conv1D(128, 3, padding="same", activation="relu"))
# model.add(MaxPooling1D(pool_size=(2)))
# model.add(Flatten())
# model.add(Dense(128, activation="relu"))
# model.add(Dropout(0.5))
# model.add(Dense(output_shape, activation="softmax"))

# # define optimizer and objective, compile cnn
# model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=64, callbacks=[model_checkpoint_callback], shuffle=True)

## CNN6

In [154]:
# ### CNN6
# model = Sequential()
# model.add(Conv1D(16, 3, padding="same",activation="relu",input_shape=(input_shape, 1)))
# model.add(Conv1D(64, 3, padding="same", activation="relu"))
# model.add(MaxPooling1D(pool_size=(2)))
# model.add(Conv1D(128, 3, padding="same", activation="relu"))
# model.add(MaxPooling1D(pool_size=(2)))
# model.add(Conv1D(256, 3, padding="same", activation="relu"))
# model.add(MaxPooling1D(pool_size=(2)))
# model.add(Conv1D(256, 3, padding="same", activation="relu"))
# model.add(MaxPooling1D(pool_size=(2)))
# model.add(Flatten())
# model.add(Dense(256, activation="relu"))
# model.add(Dropout(0.5))
# model.add(Dense(output_shape, activation="softmax"))

# # define optimizer and objective, compile cnn
# model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=64, callbacks=[model_checkpoint_callback], shuffle=True)

## CNN + LSTM

In [155]:
# ### LSTM
# model = Sequential()
# model.add(LSTM(2, input_shape=(input_shape, 1), return_sequences=True, activation='sigmoid'))
# model.add(LSTM(units=120, recurrent_activation='hard_sigmoid', activation='sigmoid'))
# model.add(Dense(activation='sigmoid', units=output_shape))
# adam=Adam(learning_rate=0.01)

# model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

# early_stop = kcallbacks.EarlyStopping(monitor='val_accuracy', patience=2, verbose=1, mode='auto')
# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=64, callbacks=[model_checkpoint_callback], shuffle=True)

# # loss: 0.4802 - accuracy: 0.7314

# LSTM

## LSTM 2

In [156]:
# ## LSTM 2
# model = Sequential()
# model.add(LSTM(4,input_shape=(input_shape, 1)))
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=64, callbacks=[model_checkpoint_callback], shuffle=True)

# # loss: 0.4497 - accuracy: 0.7483

## LSTM 3

In [157]:
# ### LSTM 3
# model = Sequential()
# model.add(LSTM(8,input_dim=input_shape, return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(LSTM(8,input_dim=input_shape, return_sequences=False))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=64, callbacks=[model_checkpoint_callback], shuffle=True)

## LSTM 4

In [158]:
# ### LSTM 4
# model = Sequential()
# model.add(LSTM(16,input_dim=input_shape, return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(LSTM(16, return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(LSTM(16, return_sequences=False))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=64, callbacks=[model_checkpoint_callback], shuffle=True)

## LSTM 5

In [159]:
# ### LSTM 5
# model = Sequential()
# model.add(LSTM(64,input_dim=input_shape, return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(LSTM(64,return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(LSTM(64, return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(LSTM(64, return_sequences=False))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=64, callbacks=[model_checkpoint_callback], shuffle=True)

# GRU

## GRU 1

In [160]:
# ### GRU 1
# model = Sequential()
# model.add(GRU(4,input_shape=(input_shape, 1)))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=32, callbacks=[model_checkpoint_callback], shuffle=True)

# # loss: 0.6380 - accuracy: 0.6299

## GRU 2

In [161]:
# ### GRU 2
# model = Sequential()
# model.add(GRU(8,input_shape=(input_shape, 1), return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(GRU(8,input_shape=(input_shape, 1), return_sequences=False))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=30, batch_size=64, callbacks=[model_checkpoint_callback], shuffle=True)
# # loss: 0.4454 - accuracy: 0.7774

## GRU 3


In [162]:
# ### GRU 3
# model = Sequential()
# model.add(GRU(16,input_shape=(input_shape, 1), return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(GRU(16, return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(GRU(16, return_sequences=False))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=30, batch_size=64, callbacks=[model_checkpoint_callback], shuffle=True)
# # loss: 0.3558 - accuracy: 0.8156

## GRU 4

In [163]:
# ### GRU 4
# model = Sequential()
# model.add(GRU(64,input_shape=(input_shape, 1), return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(GRU(64,return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(GRU(64, return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(GRU(64, return_sequences=False))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=30, batch_size=64, callbacks=[model_checkpoint_callback], shuffle=True)
# # loss: 0.2769 - accuracy: 0.8616

# SimpleRNN

## SimpleRNN 1

In [164]:
# ### SimpleRNN 1
# model = Sequential()
# model.add(SimpleRNN(4,input_shape=(input_shape, 1)))
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=32, callbacks=[model_checkpoint_callback], shuffle=True)

# # loss: 0.5145 - accuracy: 0.7150

## SimpleRNN 2

In [165]:
# ### SimpleRNN 2
# model = Sequential()
# model.add(SimpleRNN(8,input_shape=(input_shape, 1), return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(SimpleRNN(8,input_shape=(input_shape, 1), return_sequences=False))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=30, batch_size=64, callbacks=[model_checkpoint_callback], shuffle=True)
# # loss: 0.4497 - accuracy: 0.7519

## SimpleRNN 3

In [166]:
# ### SimpleRNN 3
# model = Sequential()
# model.add(SimpleRNN(16,input_shape=(input_shape, 1), return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(SimpleRNN(16, return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(SimpleRNN(16, return_sequences=False))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=30, batch_size=64, callbacks=[model_checkpoint_callback], shuffle=True)
# #loss: 0.3500 - accuracy: 0.8229

## Simple RNN 4

In [167]:
# ### Simple RNN 4
# model = Sequential()
# model.add(SimpleRNN(64,input_shape=(input_shape, 1), return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(SimpleRNN(64,return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(SimpleRNN(64, return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(SimpleRNN(64, return_sequences=False))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=30, batch_size=64, callbacks=[model_checkpoint_callback], shuffle=True)
# # loss: 0.6376 - accuracy: 0.6803

# DNN

In [168]:
# ### DNN
# model = Sequential()
# model.add(Dense(1024,input_dim=input_shape,activation='relu'))  
# model.add(Dropout(0.01))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=32, callbacks=[model_checkpoint_callback], shuffle=True)

# # loss: 0.2518 - accuracy: 0.8580

# Evaluate on test data

In [169]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=32)
print("test loss, test acc:", results)

Evaluate on test data
177/177 [==============================] - 0s 2ms/step - loss: 0.0793 - accuracy: 0.9799
test loss, test acc: [0.07925476133823395, 0.9798586368560791]


1.Evaluate on test data - csic2010 - CNN1

loss: 0.4856 - accuracy: 0.7136
test loss, test acc: [0.4856237769126892, 0.7135535478591919]


2.Evaluate on test data - fwaf -CNN1

loss: 0.0083 - accuracy: 0.9999
test loss, test acc: [0.008308237418532372, 0.9998525977134705]

3.Evaluate on test data - http - CNN1

loss: 0.0793 - accuracy: 0.9799
test loss, test acc: [0.07925476133823395, 0.9798586368560791]

#CNN2

1.Evaluate on test data - test_csic2010 - 100 epoch - CNN2

loss: 0.5204 - accuracy: 0.7044
test loss, test acc: [0.5203506946563721, 0.7044419050216675]

2.Evaluate on test data - test_fwaf - 100 epoch - CNN2

loss: 0.0103 - accuracy: 0.9997
test loss, test acc: [0.0102695869281888, 0.9996560215950012]


3.Evaluate on test data - test_http_param - 100 epoch - CNN2

loss: 0.0961 - accuracy: 0.9767
test loss, test acc: [0.09612172096967697, 0.9766784310340881]
